In [3]:
import pandas
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [6]:
cart_df = pd.read_csv("..//wheel_cart_teensy.csv")

In [7]:
cart_df

,sys_time,e_fr,e_fl,e_rr,e_rl
0,2022-06-09 18:15:44.226167,-470,-1722,897,-6141
1,2022-06-09 18:15:44.396577,-470,-1722,897,-6141
2,2022-06-09 18:15:44.396577,-470,-1722,897,-6141
3,2022-06-09 18:15:44.397077,-470,-1722,897,-6141
4,2022-06-09 18:15:44.397077,-470,-1722,897,-6141
...,...,...,...,...,...
7873,2022-06-09 18:16:55.309908,8880,-7424,-6659,2448
7874,2022-06-09 18:16:55.310408,8880,-7424,-6659,2448
7875,2022-06-09 18:16:55.310408,8880,-7424,-6659,2448
7876,2022-06-09 18:16:55.310908,8880,-7424,-6659,2448


In [12]:
mc = pd.read_csv("..//mocap_data.csv", skiprows=6)

In [14]:
st_time_mc = "2022-06-09 06.15.20.583 PM"

In [17]:
mc = mc.rename(columns={"Time (Seconds)":"seconds"})

In [18]:
mc

,Frame,seconds,center_x,center_y,center_z,org_x,org_y,org_z,xvec_x,xvec_y,xvec_z,zvec_x,zvec_y,zvec_z
0,0,0.00,0.003236,0.067702,-0.139123,-0.037691,0.065555,-0.203663,0.040555,0.064554,-0.203653,-0.039944,0.067606,-0.075845
1,1,0.01,0.003246,0.067713,-0.139121,-0.037848,0.065173,-0.203597,0.040547,0.064546,-0.203648,-0.039949,0.067603,-0.075850
2,2,0.02,0.003251,0.067696,-0.139129,-0.037697,0.065541,-0.203672,0.040666,0.064750,-0.203737,-0.039946,0.067590,-0.075851
3,3,0.03,0.003240,0.067709,-0.139121,-0.037702,0.065550,-0.203678,0.040547,0.064553,-0.203648,-0.039948,0.067594,-0.075842
4,4,0.04,0.003244,0.067717,-0.139121,-0.037855,0.065084,-0.203598,0.040558,0.064545,-0.203640,-0.039948,0.067595,-0.075849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9254,9254,92.54,0.039232,0.071813,0.051145,-0.001136,0.068630,-0.014420,0.077186,0.068300,-0.012831,-0.005280,0.070822,0.113676
9255,9255,92.55,0.039225,0.071808,0.051153,-0.001097,0.068626,-0.014390,0.077171,0.068299,-0.012826,-0.005282,0.070847,0.113693
9256,9256,92.56,0.039230,0.071796,0.051143,-0.001104,0.068610,-0.014394,0.077188,0.068304,-0.012824,-0.005281,0.070832,0.113681
9257,9257,92.57,0.039229,0.071813,0.051151,-0.001111,0.068638,-0.014422,0.077169,0.068288,-0.012831,-0.005277,0.070810,0.113667
